In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
import yaml

In [ ]:
#| hide
def load_config(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

In [ ]:
#| hide
cfg = load_config('cfg.yaml')

In [ ]:
#| hide
import os
from omegaconf import OmegaConf
cfg = OmegaConf.create(cfg)
cfg.data.name = 'MNIST'
from datetime import datetime
cfg.save_dir = os.path.join(cfg.save_dir, 
                            datetime.now().strftime("%Y%m%d_%H%M%S"))

In [ ]:
import os
import torch.nn as nn
import torch
from fedai.vision.VisionBlock import VisionBlock
from fedai.utils import * # noqa: F403

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, dim_in, dim_hidden, dim_out):
        super(MLP, self).__init__()
        self.layer_input = nn.Linear(dim_in, dim_hidden)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout()
        self.layer_hidden = nn.Linear(dim_hidden, dim_out)

    def forward(self, x):
        x = x.view(-1, x.shape[1]*x.shape[-2]*x.shape[-1])
        x = self.layer_input(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.layer_hidden(x)
        return x

In [ ]:
def get_block(cfg, id, train=True):
    block = VisionBlock if cfg.data.modality == ['Vision'] else None
    return block(cfg, id, train=train)

In [ ]:
def client_fn(client_cls, cfg, id, latest_round):
    model = MLP(28*28, 128, 10)
    criterion = torch.nn.CrossEntropyLoss()
    train_block = get_block(cfg, id)
    test_block = get_block(cfg, id, train=False)
    state = {'model': model, 'optimizer': None, 'criterion': criterion}

    if id in latest_round:
        comm_round = latest_round[id]
        state['model'] = load_state_from_disk(cfg, model, id, comm_round)
    
    return client_cls(id, cfg, state, block= [train_block, test_block])


In [ ]:
from fedai.federated.agents import * # noqa: F403
from fedai.client_selector import *  # noqa: F403
from torch.nn.modules import CrossEntropyLoss # noqa: F403

client_selector = BaseClientSelector(cfg)  # noqa: F405
client_cls = FLAgent # noqa: F405
server = client_cls(cfg= cfg, block= None, id= 0, state= None, role= AgentRole.SERVER)
latest_round = {}

In [ ]:
# from fedai.trainers import *
# all_ids = client_selector.select()
# for t in range(1, 3):
#     lst_active_ids = all_ids[t]
#     len_clients_ds = []
#     for id in lst_active_ids:
#         client = client_fn(client_cls, cfg, id, latest_round)
#         len_clients_ds.append(200)
#         server.communicate(client, t) # read from the disk
#         trainer = Trainer(client) # the trainer object takes a client and make local training on its dataset.
#         client_history = trainer.train() # actual training loop
#         client.communicate(server, t) # save the state of the client to the disk
#         latest_round[id] = t # make sure you tell the client_fn where to look
#     server.aggregate(lst_active_ids, t, len_clients_ds) # aggregate the models of the clients


data/MNIST/train data/MNIST/test

Dataset already generated.

data/MNIST/train data/MNIST/test

Dataset already generated.



  0%|          | 0/484 [08:34<?, ?it/s]


data/MNIST/train data/MNIST/test

Dataset already generated.

data/MNIST/train data/MNIST/test

Dataset already generated.



  0%|          | 0/513 [08:43<?, ?it/s]
/home/ahmed/Ahmed-home/1- Projects/fedai/fedai/federated/agents.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  


AttributeError: 'FLAgent' object has no attribute 'device'